In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("../DataFiles/LeCunn_TrainDataNormPCA_ILW.csv")

print (df)

# print (df)
classificationArray = list(df.Class)
print(len(classificationArray))
# print (classificationArray)
pc1Array = np.array(df.First)
pc2Array = np.array(df.Second)

# print(pc1Array)
# print(pc2Array)
pc1Max = pc1Array.max()
print("pc1Max",pc1Max)
pc2Max = pc2Array.max()
print("pc2Max",pc2Max)
pc1Min = pc1Array.min()
print("pc1Min", pc1Min)
pc2Min = pc2Array.min()
print("pc2Min", pc2Min)

numISamples = list(classificationArray).count("I")
print("numISamples",numISamples)
numLSamples = list(classificationArray).count("L")
print("numLSamples", numLSamples)
numWSamples = list(classificationArray).count("W")
print("numWSamples", numWSamples)

numberOfBins = np.log2(numISamples).astype('int32')
print ("numberOfBins", numberOfBins)

iHistogram, lHistogram, wHistogram = build2DHistogramClassifier(numberOfBins, pc1Array, pc2Array, classificationArray)

df1 = pd.read_csv("../DataFiles/LeCunn_TestDataNormPCA_ILW.csv")
testPc1Array = np.array(df1.First)
testPc2Array = np.array(df1.Second)
testClassificationArray = list(df1.Class)

result = query (iHistogram, lHistogram, wHistogram, pc1Max, pc2Max, pc1Min, pc2Min, numberOfBins, testPc1Array, testPc2Array)
print (len(result))
# print(result)

correct = 0
wrong = 0
for i, j in zip(result, testClassificationArray) :
    if (i == j):
        correct += 1
    else :
        wrong += 1
print (correct)
print (wrong)
print (correct / (correct + wrong))
confusionMatrix = np.zeros([3,3])
for (i, j) in zip (result, testClassificationArray):
    row = -1
    col = -1
    if (i == "I"):
        row = 0
    elif (i == "L"):
        row = 1
    elif (i == "W"):
        row = 2
    
    if (j == "I"):
        col = 0
    elif (j == "L"):
        col = 1
    elif (j == "W"):
        col = 2
    if (row != -1 or col != -1) :
        confusionMatrix[row][col] +=1

print(confusionMatrix)
# print (sum(iHistogram))

print ("iHistogram\n",iHistogram)
print("lHistogram\n",lHistogram)
print("wHistogram\n", wHistogram)

     Class     First    Second     Third         4         5         6  \
0        I -0.151875 -1.736774  1.579057  2.264171 -1.073543 -0.968191   
1        W -5.743487 -0.636573 -1.428790 -0.889334 -0.237958 -0.631674   
2        L  3.605518 -0.229735 -3.442588  0.103254 -0.387951  0.572051   
3        L  3.049257  0.294185  1.556032  0.583538  0.074783  1.767507   
4        W -2.471456 -0.410800 -0.363233  1.100983  0.203281 -0.264028   
5        L  3.225230 -0.622863 -3.783401  0.325517 -0.458438 -0.245207   
6        L  0.379102 -3.148988  1.149021  0.886711 -0.228640 -0.112329   
7        L  3.746336 -0.151511 -3.398074  0.131155 -0.398510  0.494286   
8        W -0.808883  3.022032 -0.081961  0.716804  0.163274 -0.543833   
9        W -1.445279  1.199717 -0.648030  0.644548 -1.121323 -0.250069   
10       W  0.237647  3.689478 -0.274190  1.313088 -0.058507  0.590472   
11       I  0.785746 -0.494874  2.665020 -0.871918 -0.791197 -0.720561   
12       I  0.420834 -0.680930  2.1312

NameError: name 'build2DHistogramClassifier' is not defined

In [ ]:
def query (histI, histL, histW, pc1max, pc2max, pc1min, pc2min, bincount, pc1, pc2) :
    results = list()
    for i, j in zip (pc1, pc2):
        x = CalculateBin(i, pc1max, pc1min, bincount)
        y = CalculateBin(j, pc2max, pc2min, bincount)
        histIValue = histI[x][y]
        histLValue = histL[x][y]
        histWValue = histW[x][y]
        probI = histIValue / (histIValue + histLValue + histWValue)
        probL = histLValue / (histIValue + histLValue + histWValue)
        probW = histWValue / (histIValue + histLValue + histWValue)
        largestProb = max (probI, probL, probW)
        if(largestProb == probI):
            results.append("I")
        elif (largestProb == probL):
            results.append("L")
        elif (largestProb == probW):
            results.append("W")
        else :
            results.append("Undefined")
    return results
            

In [ ]:
def histogramQueryEr (QueryMatrix, mu, eigen1, eigen2, pc1max, pc1min, pc2max, pc2min, bincount, iHist, lHist, wHist) :
    results = list()
    for queries in QueryMatrix :
        p1test, p2test, z = convertQuery(queries, mu, eigen1, eigen2)
        popo = queryHistogram(p1test, p2test, posClassi, pc1max, pc2max, pc1min, pc2min, bincount)
        negneg = queryHistogram(p1test, p2test, negClassi, pc1max, pc2max, pc1min, pc2min, bincount)
        if (popo / (popo+negneg) > negneg / (popo+negneg)) :
            results.append(8)
        else :
            results.append(1)
    return results

In [ ]:
def build2DHistogramClassifier(bincount, pc1, pc2,
                               classlabels):
    pc1max = pc1.max()
    pc2max = pc2.max()
    pc1min = pc1.min()
    pc2min = pc2.min()

    iClass = np.zeros(shape=(bincount,bincount)).astype('int32')
    lClass = np.zeros(shape=(bincount,bincount)).astype('int32')
    wClass = np.zeros(shape=(bincount,bincount)).astype('int32')
    
    for i, j, k in zip (pc1, pc2, classlabels) :
        
        row = CalculateBin(i, pc1max, pc1min, bincount)
        col = CalculateBin(j, pc2max, pc2min, bincount)
        if(k == "I"):
            iClass[row][col] += 1
        elif (k == "L"):
            lClass[row][col] += 1
        elif (k == "W"):
            wClass[row][col] += 1

            
    return iClass, lClass, wClass

In [2]:
def CalculateBin (FeatureValue, FeatureMax, FeatureMin, NumberOfBins):
    binindex = (np.round(((NumberOfBins-1)*(FeatureValue-FeatureMin)/
                          (FeatureMax-FeatureMin)))).astype('int32')
    return binindex